In [27]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import polars as pl
from sklearn.metrics import average_precision_score
from tqdm import tqdm

In [37]:
metadata = pd.read_csv("Enformer_metadata.csv")
metadata["name2"] = "feature_" + metadata.index.astype(str)
metadata

,name,file,clip,scale,sum_stat,description,assay,sample,name2
0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...,DNASE,cerebellum male adult (27 years) and male adul...,feature_0
1,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...,DNASE,frontal cortex male adult (27 years) and male ...,feature_1
2,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion,DNASE,chorion,feature_2
3,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...,DNASE,Ishikawa treated with 0.02% dimethyl sulfoxide...,feature_3
4,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348,DNASE,GM03348,feature_4
...,...,...,...,...,...,...,...,...,...
5308,CNhs14239,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R,CAGE,epithelioid sarcoma cell line:HS-ES-2R,feature_5308
5309,CNhs14240,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:squamous cell lung carcinoma cell line:RE...,CAGE,squamous cell lung carcinoma cell line:RERF-LC-AI,feature_5309
5310,CNhs14241,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:gastric cancer cell line:GSS,CAGE,gastric cancer cell line:GSS,feature_5310
5311,CNhs14244,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:carcinoid cell line:NCI-H727,CAGE,carcinoid cell line:NCI-H727,feature_5311


In [29]:
dataset_path = "gonzalobenegas/dsqtl"
V = load_dataset(dataset_path, split="test").to_pandas()
V

,chrom,pos,ref,alt,label
0,1,817416,C,T,False
1,1,906870,A,G,False
2,1,910558,G,A,False
3,1,911109,T,C,False
4,1,911661,C,T,False
...,...,...,...,...,...
28271,22,50636975,C,A,False
28272,22,50704264,T,A,False
28273,22,50718505,A,G,False
28274,22,50739500,G,A,False


In [30]:
features = "Enformer_absDelta"
df = pd.read_parquet(f"https://huggingface.co/datasets/{dataset_path}/resolve/main/features/{features}.parquet")
feature_cols = df.columns
V = pd.concat([V, df], axis=1)
V = V.dropna()
V

,chrom,pos,ref,alt,label,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_5303,feature_5304,feature_5305,feature_5306,feature_5307,feature_5308,feature_5309,feature_5310,feature_5311,feature_5312
0,1,817416,C,T,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,906870,A,G,False,0.000684,0.000629,0.001203,0.001003,0.001418,...,0.000076,0.000451,0.000123,0.000091,1.707077e-04,0.000124,0.000063,0.000084,0.000233,0.000056
2,1,910558,G,A,False,0.000046,0.000088,0.000183,0.000004,0.000058,...,0.000032,0.000016,0.000001,0.000004,7.152557e-07,0.000003,0.000009,0.000008,0.000045,0.000019
3,1,911109,T,C,False,0.000053,0.000004,0.000314,0.000143,0.000146,...,0.000134,0.000100,0.000017,0.000011,3.343821e-05,0.000088,0.000059,0.000018,0.000087,0.000112
4,1,911661,C,T,False,0.000037,0.000030,0.000007,0.000025,0.000013,...,0.000001,0.000010,0.000004,0.000002,2.229214e-05,0.000007,0.000004,0.000017,0.000046,0.000067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28271,22,50636975,C,A,False,0.000031,0.000056,0.000073,0.000026,0.000015,...,0.000593,0.000260,0.000184,0.000097,1.358986e-05,0.000049,0.000069,0.000008,0.000087,0.000121
28272,22,50704264,T,A,False,0.000021,0.000008,0.000023,0.000015,0.000024,...,0.000152,0.000297,0.000088,0.000064,6.854534e-06,0.000010,0.000007,0.000002,0.000005,0.000007
28273,22,50718505,A,G,False,0.000024,0.000033,0.000232,0.000014,0.000027,...,0.000300,0.000085,0.000010,0.000009,7.748604e-06,0.000005,0.000002,0.000009,0.000088,0.000062
28274,22,50739500,G,A,False,0.000017,0.000069,0.000096,0.000038,0.000002,...,0.000146,0.000115,0.000035,0.000001,3.683567e-05,0.000043,0.000050,0.000016,0.000213,0.000142


In [39]:
res = []
for c in tqdm(feature_cols):
    res.append([c, average_precision_score(V["label"], V[c])])
res = pd.DataFrame(res, columns=["feature", "AUPRC"]).sort_values("AUPRC", ascending=False)
res

100%|███████████████████████████████████████████████████████| 5313/5313 [00:29<00:00, 178.49it/s]


,feature,AUPRC
18,feature_18,0.497530
1267,feature_1267,0.489108
12,feature_12,0.485325
16,feature_16,0.475115
11,feature_11,0.473027
...,...,...
5004,feature_5004,0.058887
4684,feature_4684,0.058122
4896,feature_4896,0.057867
4974,feature_4974,0.057359


In [40]:
res = res.merge(metadata, left_on="feature", right_on="name2")
res

,feature,AUPRC,name,file,clip,scale,sum_stat,description,assay,sample,name2
0,feature_18,0.497530,ENCFF814MPK,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM19240,DNASE,GM19240,feature_18
1,feature_1267,0.489108,ENCFF782WWH,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:SMC3:GM12878,CHIP,SMC3:GM12878,feature_1267
2,feature_12,0.485325,ENCFF093VXI,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM12878,DNASE,GM12878,feature_12
3,feature_16,0.475115,ENCFF385ITM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM19238,DNASE,GM19238,feature_16
4,feature_11,0.473027,ENCFF149TAY,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM10248,DNASE,GM10248,feature_11
...,...,...,...,...,...,...,...,...,...,...,...
5308,feature_5004,0.058887,CNhs11837,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:transitional cell carcinoma cell line:Hs ...,CAGE,transitional cell carcinoma cell line:Hs 769.T,feature_5004
5309,feature_4684,0.058122,CNhs10621,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,"CAGE:heart, adult, pool1",CAGE,"heart, adult, pool1",feature_4684
5310,feature_4896,0.057867,CNhs11348,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,"CAGE:Fibroblast - Mammary,",CAGE,"Fibroblast - Mammary,",feature_4896
5311,feature_4974,0.057359,CNhs11789,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,"CAGE:left ventricle, adult,",CAGE,"left ventricle, adult,",feature_4974


In [41]:
# GM19240 is LCL from B-Lymphocyte, Yoruban
res[["AUPRC", "description"]].head()

,AUPRC,description
0,0.497530,DNASE:GM19240
1,0.489108,CHIP:SMC3:GM12878
2,0.485325,DNASE:GM12878
3,0.475115,DNASE:GM19238
4,0.473027,DNASE:GM10248


In [42]:
res.groupby("assay").AUPRC.mean().sort_values(ascending=False)

assay
ATAC     0.203169
DNASE    0.184816
CHIP     0.156486
CAGE     0.073068
Name: AUPRC, dtype: float64